In [20]:
import pyaudio
import numpy as np

import math

p = pyaudio.PyAudio()

volume = 0.5     # range [0.0, 1.0]
fs = 44100       # sampling rate, Hz, must be integer
duration = 5.0   # in seconds, may be float 

In [12]:
def play_frequencies(p, fs, volume, freq = [440,]):
    """Found most of this method online at 
    https://stackoverflow.com/questions/8299303/generating-sine-wave-sound-in-python"""
    for f in freq:
        # generate samples, note conversion to float32 array
        samples = (np.sin(2*np.pi*np.arange(fs*duration)*f/fs)).astype(np.float32)

        # for paFloat32 sample values must be in range [-1.0, 1.0]
        stream = p.open(format=pyaudio.paFloat32,
                        channels=1,
                        rate=fs,
                        output=True)

        # play. May repeat with different volume values (if done interactively) 
        stream.write(volume*samples)

    stream.stop_stream()
    stream.close()

    #p.terminate()

In [23]:
# test
play_frequencies(p, fs, volume, freq = [300,10*300])

In [24]:
# equal tempered scale (or how keyboard players get deceived into thinking they know something about music)
# equations from https://pages.mtu.edu/~suits/NoteFreqCalcs.html

def equal_tempered(base_frequency, halfsteps_away = [0,], halfsteps_in_a_scale = 12):
    """Parameters:
            base_frequency : float
                scale will start from this frequency, default A1 (440 Hz)
                
            halfsteps_away : list of integers
                list of number of halfsteps between the resulting and the base frequency
                
            halfsteps_in_a_scale : integer
                number of halfsteps between the nearest octaves
    """
    frequencies = []
    for halfstep_count in halfsteps_away:
        freq = base_frequency * (2**(1/halfsteps_in_a_scale))**halfstep_count
        frequencies.append(freq)
        
    return frequencies

# just scale occurs naturally as a result of the overtone series for simple systems such as vibrating strings or air columns
# source: https://pages.mtu.edu/~suits/scales.html

def just_scale(base_frequency, halfsteps_away = [0,]):
    """Parameters:
            base_frequency : float
                scale will start from this frequency, default A1 (440 Hz)
                
            halfsteps_away : list of integers
                list of number of halfsteps between the resulting and the base frequency
    """
    harmonics = [
        base_frequency * 1.,
        base_frequency * 25./24.,
        base_frequency * 9./8.,
        base_frequency * 6./5.,
        base_frequency * 5./4.,
        base_frequency * 4./3.,
        base_frequency * 45./32.,
        # perfect fifth
        base_frequency * 3./2.,
        base_frequency * 8./5.,
        base_frequency * 5./3.,
        base_frequency * 9./5.,
        base_frequency * 15./8.,
        base_frequency * 2.,
    ]
    
    frequencies = []
    for halfstep_count in halfsteps_away:
        freq = harmonics[halfstep_count]
        frequencies.append(freq)
        
    return frequencies

def perfect_fifths(base_frequency, n = 12):
    """I made this one up (I'm sure it already exists.)
    
    Parameters:
            base_frequency : float
                scale will start from this frequency, default A1 (440 Hz)
                
            n : integer, keep it at 12...
                count of 3/2 (perfect fifths) to step through and construct the frequencies
                for the notes that are within the same octave as the base_frequency
    """
    freq = []
    for i in range(n):
        f = base_frequency * (3./2.)**i/(2.**int(math.log2((3/2.)**i)))
        freq.append(f)
        
    if n == 12:
        freq.append(base_frequency * 2)
        
    freq.sort()

    return freq

In [26]:
def make_scale(kind = 'minor', base_frequency = 440, halfsteps_in_a_scale = 12, tuning = 'equal_tempered'):
    """Parameters:
            kind : str
                'minor'
                'major (melodic up, natural down)'
                'harmonics'
                
            base_frequency : float
                scale will start from this frequency, default A1 (440 Hz)
                
            halfsteps_in_a_scale : integer
                number of halfsteps between the nearest octaves
                
            tuning : str
                'equal_tempered'
                'just_scale'
                'perfect_fifths'
    """
    if kind == 'harmonics':
        halfsteps_list_up = range(halfsteps_in_a_scale+1)
        halfsteps_list_down = halfsteps_list_up[::-1]
    elif kind == 'major':
        if halfsteps_in_a_scale !=12:
            raise ValueError
        else:
            halfsteps_list_up = [0, 2, 4, 5, 7, 9, 11, 12]
            halfsteps_list_down = halfsteps_list_up[::-1]
    elif kind == 'minor':
        if halfsteps_in_a_scale !=12:
            raise ValueError
        else:
            halfsteps_list_up = [0, 2, 3, 5, 7, 9, 11, 12]
            halfsteps_list_down = [0, 2, 3, 5, 7, 8, 10, 12][::-1]
            
    if tuning == 'equal_tempered':
        frequencies_up = equal_tempered(
                            base_frequency, 
                            halfsteps_away = halfsteps_list_up, 
                            halfsteps_in_a_scale = halfsteps_in_a_scale)

        frequencies_down = equal_tempered(
                        base_frequency, 
                        halfsteps_away = halfsteps_list_down, 
                        halfsteps_in_a_scale = halfsteps_in_a_scale)
        
    if tuning == 'just_scale':
        if halfsteps_in_a_scale !=12:
            raise ValueError
        else:
            frequencies_up = just_scale(
                                base_frequency, 
                                halfsteps_away = halfsteps_list_up)
            frequencies_down = just_scale(
                                base_frequency, 
                                halfsteps_away = halfsteps_list_down)
            
    if tuning == 'perfect_fifths':
        if halfsteps_in_a_scale !=12:
            raise ValueError
        else:
            frequencies = perfect_fifths(
                                base_frequency, 
                                n = 12)

            frequencies_up = []
            frequencies_down = []
            
            for indx in halfsteps_list_up:
                freq_up = frequencies[indx]
                frequencies_up.append(freq_up)
            for indx in halfsteps_list_down:
                freq_dn = frequencies[indx]
                frequencies_down.append(freq_dn)
        
    up_then_down = frequencies_up + frequencies_down[1:]
    
    return frequencies_up, frequencies_down, up_then_down

In [27]:
duration = 1.0

# sine frequency, Hz, may be float

base_frequency_C4 = 261.63
# base_frequency_D4 = 293.66
# base_frequency_G4 = 392.00
# base_frequency_A4 = 440 #Hz

# base_frequency = base_frequency_A4
base_frequency = 261.63

# Equal tempered

In [30]:
duration = 2.

# harmonics (12 half steps between two octaves, with an octave being 2*f)
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='harmonics', base_frequency = base_frequency, tuning = 'equal_tempered')

play_frequencies(p, fs, volume, freq = up_then_down)

In [32]:
# major scale
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='major', base_frequency = base_frequency, tuning = 'equal_tempered')

play_frequencies(p, fs, volume, freq = up_then_down)
up_then_down

[261.63,
 293.66974569918125,
 329.6331442839957,
 349.23415104650616,
 392.0020805232463,
 440.0074582456588,
 493.8916728538232,
 523.2600000000002,
 493.8916728538232,
 440.0074582456588,
 392.0020805232463,
 349.23415104650616,
 329.6331442839957,
 293.66974569918125,
 261.63]

In [33]:
# minor scale
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='minor', base_frequency = base_frequency, tuning = 'equal_tempered')

play_frequencies(p, fs, volume, freq = up_then_down)

# [Just scale](https://pages.mtu.edu/~suits/overtone.html)

In [34]:
duration = 1.

# harmonics (12 half steps between two octaves, with an octave being 2*f)
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='harmonics', base_frequency = base_frequency, tuning = 'just_scale')

play_frequencies(p, fs, volume, freq = up_then_down)

In [35]:
# major scale
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='major', base_frequency = base_frequency, tuning = 'just_scale')

play_frequencies(p, fs, volume, freq = up_then_down)
up_then_down

[261.63,
 294.33375,
 327.0375,
 348.84,
 392.445,
 436.05,
 490.55625,
 523.26,
 490.55625,
 436.05,
 392.445,
 348.84,
 327.0375,
 294.33375,
 261.63]

In [36]:
# minor scale
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='minor', base_frequency = base_frequency, tuning = 'just_scale')

play_frequencies(p, fs, volume, freq = up_then_down)

# Perfect Fifths

In [37]:
duration = 1.

# harmonics (12 half steps between two octaves, with an octave being 2*f)
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='harmonics', base_frequency = base_frequency, tuning = 'perfect_fifths')

play_frequencies(p, fs, volume, freq = up_then_down)

In [38]:
# harmonics (12 half steps between two octaves, with an octave being 2*f)
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='major', base_frequency = base_frequency, tuning = 'perfect_fifths')

play_frequencies(p, fs, volume, freq = up_then_down)
up_then_down

[261.63,
 294.33375,
 331.12546875,
 353.59931648254394,
 392.445,
 441.500625,
 496.688203125,
 523.26,
 496.688203125,
 441.500625,
 392.445,
 353.59931648254394,
 331.12546875,
 294.33375,
 261.63]

In [39]:
# harmonics (12 half steps between two octaves, with an octave being 2*f)
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='minor', base_frequency = base_frequency, tuning = 'perfect_fifths')

play_frequencies(p, fs, volume, freq = up_then_down)

# Experimental 

In [40]:
# what if there were more halfsteps in a scale

duration = .5

# harmonics
frequencies_up, frequencies_down, up_then_down = make_scale(
    kind='harmonics', base_frequency = base_frequency, halfsteps_in_a_scale = 20, tuning = 'equal_tempered')

play_frequencies(p, fs, volume, freq = up_then_down)

In [41]:
# overtone series
# https://pages.mtu.edu/~suits/overtone.html

duration = 3.5
base_frequency = 131

play_frequencies(p, fs, volume, freq = [base_frequency, 2*base_frequency, 
                                        3*base_frequency, 4*base_frequency, 
                                        5*base_frequency, 6*base_frequency, 
                                        7*base_frequency,
                                        # just for fun
                                        8*base_frequency,
                                        9*base_frequency, 10*base_frequency,
                                        11*base_frequency, 12*base_frequency])

In [14]:
# harmonics
[1., 25./24., 9./8., 6./5., 5./4., 4./3., 45./32., 3./2., 8./5., 5./3., 9./5., 15./8., 2.]

[1.0,
 1.0416666666666667,
 1.125,
 1.2,
 1.25,
 1.3333333333333333,
 1.40625,
 1.5,
 1.6,
 1.6666666666666667,
 1.8,
 1.875,
 2.0]